In [1]:
import os
import matplotlib.pyplot as plt
from itertools import combinations
import numpy as np
from PIL import Image as im
import shutil
import math
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from numpy import mean
from numpy import absolute
from numpy import sqrt
import pandas as pd
from sklearn import linear_model
import torch
import cv2
import csv
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import ndimage
from PIL import Image

torch.manual_seed(42)

In [2]:
#Create a table, 
#where each row contains information [match# with img1] [match# with img2] [match# with img384] [Class Name of this row]
path = '../Experiment2/MatchNumbersTable.npz'
number_of_matches_between_every_image = np.load(path)['arr_0']
TotalImage=number_of_matches_between_every_image.shape[0]

path = '../Experiment2/2D_list_of_all_npzs_with_class.npz'
name_of_npzfile_between_each_pair_with_className_of_rowImg = np.load(path)['arr_0']
class_name_of_each_rowImg=name_of_npzfile_between_each_pair_with_className_of_rowImg[:,TotalImage]

class_names=list(set(class_name_of_each_rowImg))
class_name_of_each_rowImg=np.reshape(class_name_of_each_rowImg,(TotalImage,1))
number_of_matches_between_every_image_with_className=np.append(number_of_matches_between_every_image,class_name_of_each_rowImg,1)


In [8]:
output_path=current_dir+"/output/"
if not os.path.exists(output_path):
    os.makedirs(output_path)
    
instruction="../match_pairs.py --resize 1600 --superglue outdoor --max_keypoints 2048 --nms_radius 3  --resize_float \
            --input_dir ./ --input_pairs pair.txt --output_dir " +output_path +" --viz"
print(instruction)
os.system(instruction) 

../match_pairs.py --resize 1600 --superglue outdoor --max_keypoints 2048 --nms_radius 3  --resize_float             --input_dir ./ --input_pairs pair.txt --output_dir /home/disaster/Documents/github/SuperGlue_Unsupervised/Try_Two_Images/output/ --viz
Namespace(input_pairs='pair.txt', input_dir='./', output_dir='/home/disaster/Documents/github/SuperGlue_Unsupervised/Try_Two_Images/output/', max_length=-1, resize=[1600], resize_float=True, superglue='outdoor', max_keypoints=2048, keypoint_threshold=0.005, nms_radius=3, sinkhorn_iterations=20, match_threshold=0.2, viz=True, eval=False, fast_viz=False, cache=False, show_keypoints=False, viz_extension='png', opencv_display=False, shuffle=False, force_cpu=False)
Will resize max dimension to 1600
Running inference on device "cpu"
Loaded SuperPoint model
Loaded SuperGlue model ("outdoor" weights)
Looking for data in directory "."
Will write matches to directory "/home/disaster/Documents/github/SuperGlue_Unsupervised/Try_Two_Images/output"
Will

0

In [7]:
#..................Create pair.txt file for superglue..............#
current_dir=os.getcwd()
f= open("pair.txt","w+")
file_1=""
file_2=""

for file in os.scandir(current_dir):
    if file.name.endswith(".png") and file_1=="":
        file_1=file.name
    elif file.name.endswith(".png") and file_2=="":
        file_2=file.name

print(file_1," ",file_2)
to_write=file_1+" "+file_2+"\n"
f.write(to_write)
to_write=file_1+" "+file_1+"\n"
f.write(to_write)
to_write=file_2+" "+file_2+"\n"
f.write(to_write)
f.close()

#...........Done with writing in text file........................#

#...........Run the Instruction of superglue......................#
output_path=current_dir+"/output/"
if not os.path.exists(output_path):
    os.makedirs(output_path)
    
instruction="../match_pairs.py --resize 1600 --superglue outdoor --max_keypoints 2048 --nms_radius 3  --resize_float \
            --input_dir ./ --input_pairs pair.txt --output_dir " +output_path +" --viz"
print(instruction)
os.system(instruction) 

#...............Run the keypoint showing algo for each image......................#

for file in os.scandir(output_path):
    if file.name.endswith(".npz"):
        splited_str=file.name.removesuffix("_matches.npz").split("_")
        print(splited_str)
        image_name_1=splited_str[0]
        image_name_2=splited_str[1]
        
        if image_name_1== image_name_2:
            img=Image.open(current_dir+"/"+image_name_1+".png")
            wpercent = (1600/float(img.size[0]))
            hsize = int((float(img.size[1])*float(wpercent)))
            img = img.resize((1600,hsize), resample=Image.LANCZOS)
            
            plt.clf()
            f, axarr = plt.subplots(2,1)
            axarr[1].imshow(img)

            npz=np.load(file)
            key_points=npz['keypoints0']

            for i in range(key_points.shape[0]):
                axarr[0].plot(key_points[i][0], key_points[i][1], marker='.', markersize=2,color="white")
            axarr[0].imshow(img)
            plt.savefig(output_path+"/"+image_name_1)
            plt.close()

#............. Create Heatmap..................#

for file in os.scandir(current_dir):
    if file.name.endswith(".png"):
        self_flag=False
        cross_flag=False
        
        heatmap_path=current_dir+"/output/"+file.name.removesuffix(".png")
        if not os.path.exists(heatmap_path):
            os.makedirs(heatmap_path)
        heatmap_path+="/"
        
        for npz_file in os.scandir(output_path):
            if npz_file.name.endswith(".npz"):
                
                splited_str=npz_file.name.removesuffix("_matches.npz").split("_")
                #print(splited_str)
                image_name_1=splited_str[0]
                image_name_2=splited_str[1]
                
                img=Image.open(current_dir+"/"+file.name)
                wpercent = (1600/float(img.size[0]))
                hsize = int((float(img.size[1])*float(wpercent)))
                #img = img.resize((1600,hsize), resample=Image.LANCZOS)
                
                if image_name_1== image_name_2 and image_name_1== file.name.removesuffix(".png"):
                    self_flag=True
                    
                    self_image_array=np.zeros((1600,1600))
                    npz=np.load(npz_file)
                    key_points=npz['keypoints0']
                    
                    for i in range(key_points.shape[0]):
                        self_image_array[int(key_points[i][1])][int(key_points[i][0])]+=1
                        
                    self_image_array=ndimage.uniform_filter(self_image_array, size=5, mode='constant')
                    self_image_array=self_image_array[0:hsize,:]
                    
                    plt.clf()
                    plt.subplots(figsize=(5,3))
                    sns.set()
                    ax = sns.heatmap(self_image_array,cmap='Reds')
                    plt.title("self_map of "+ file.name)
                    plt.savefig(heatmap_path+"_self")
                    plt.close()
               
                elif image_name_1!= image_name_2 and image_name_2== file.name.removesuffix(".png"):
                    cross_flag=True
                    cross_image_array=np.zeros((1600,1600))
                    npz_self_all_keypoints=np.load(output_path+"/"+image_name_2+"_"+image_name_2+"_matches.npz")
                    npz=np.load(npz_file)
                    
                    key_points=npz_self_all_keypoints['keypoints0']
                    print(key_points.shape)
                    
                    #printing only matches
                    for i in range(npz['keypoints0'].shape[0]):
                        if npz['matches'][i]>-1:
                            cross_image_array[int(npz['keypoints1'][npz['matches'][i]][1])]\
                            [int(npz['keypoints1'][npz['matches'][i]][0])]+=1
                    
                    cross_image_array=ndimage.uniform_filter(cross_image_array, size=5, mode='constant')
                    cross_image_array=cross_image_array[0:hsize,:]
                    
                    plt.clf()
                    plt.subplots(figsize=(5,3))
                    sns.set()
                    ax = sns.heatmap(cross_image_array,cmap='Blues')
                    plt.title("cross_map of "+ file.name)
                    plt.savefig(heatmap_path+"_cross")
                    plt.close()
                    
                elif image_name_1!= image_name_2 and image_name_1== file.name.removesuffix(".png"):
                    cross_flag=True
                    cross_image_array=np.zeros((1600,1600))
                    npz=np.load(npz_file)
                    key_points=npz['keypoints0']
                    
                    for i in range(key_points.shape[0]):
                        if npz['matches'][i]>-1:
                            cross_image_array[int(key_points[i][1])][int(key_points[i][0])]+=1
                        
                    cross_image_array=ndimage.uniform_filter(self_image_array, size=5, mode='constant')
                    cross_image_array=cross_image_array[0:hsize,:]
                    
                    plt.clf()
                    plt.subplots(figsize=(5,3))
                    sns.set()
                    ax = sns.heatmap(self_image_array,cmap='Blues')
                    plt.title("cross_map of "+ file.name)
                    plt.savefig(heatmap_path+"_cross")
                    plt.close()
                    
            if self_flag == True and cross_flag== True:
                diff_image_array=self_image_array-cross_image_array
                diff_image_array[diff_image_array<0]=0
                diff_image_array=ndimage.uniform_filter(diff_image_array, size=5, mode='constant')
                diff_image_array=diff_image_array[0:hsize,:]
                    
                plt.clf()
                plt.subplots(figsize=(5,3))
                sns.set()
                ax = sns.heatmap(diff_image_array,cmap='Greens')
                plt.title("diff_map of "+ file.name)
                plt.savefig(heatmap_path+"_diff")
                plt.close()
                    
                    
        plt.show()

schunkei1.png   luscombei.png
../match_pairs.py --resize 1600 --superglue outdoor --max_keypoints 2048 --nms_radius 3  --resize_float             --input_dir ./ --input_pairs pair.txt --output_dir /home/disaster/Documents/github/SuperGlue_Unsupervised/Try_Two_Images/output/ --viz
Namespace(input_pairs='pair.txt', input_dir='./', output_dir='/home/disaster/Documents/github/SuperGlue_Unsupervised/Try_Two_Images/output/', max_length=-1, resize=[1600], resize_float=True, superglue='outdoor', max_keypoints=2048, keypoint_threshold=0.005, nms_radius=3, sinkhorn_iterations=20, match_threshold=0.2, viz=True, eval=False, fast_viz=False, cache=False, show_keypoints=False, viz_extension='png', opencv_display=False, shuffle=False, force_cpu=False)
Will resize max dimension to 1600
Running inference on device "cpu"
Loaded SuperPoint model
Loaded SuperGlue model ("outdoor" weights)
Looking for data in directory "."
Will write matches to directory "/home/disaster/Documents/github/SuperGlue_Unsupervis

ValueError: operands could not be broadcast together with shapes (906,1600) (846,1600) 

<Figure size 640x480 with 0 Axes>

In [ ]:
#...............Run the keypoint showing algo for each image......................#

for file in os.scandir(output_path):
    if file.name.endswith(".npz"):
        splited_str=file.name.removesuffix("_matches.npz").split("_")
        image_name_1=splited_str[0]+"_"+splited_str[1]+"_"+splited_str[2]
        image_name_2=splited_str[3]+"_"+splited_str[4]+"_"+splited_str[5]
        
        if image_name_1== image_name_2:
            img=Image.open(current_dir+"/"+image_name_1+".png")
            wpercent = (1600/float(img.size[0]))
            hsize = int((float(img.size[1])*float(wpercent)))
            img = img.resize((1600,hsize), resample=Image.LANCZOS)
            
            plt.clf()
            f, axarr = plt.subplots(2,1)
            axarr[1].imshow(img)

            npz=np.load(file)
            key_points=npz['keypoints0']

            for i in range(key_points.shape[0]):
                axarr[0].plot(key_points[i][0], key_points[i][1], marker='.', markersize=2,color="white")
            axarr[0].imshow(img)
            plt.savefig(output_path+"/"+image_name_1)
            plt.close()

SyntaxError: invalid syntax (2110584633.py, line 1)